In [1]:
import math
import random

# 定数の定義
d0 = 1.0  # 基準距離 [m]
PL_bar = 40.0  # 平均RSSI [dBm]
gamma = 3.0  # 距離減衰定数
sigma = 2.0  # X_sigma の標準偏差 [dB]

# ガウス分布に従うランダムな値 X_sigma を生成
X_sigma = random.gauss(0, sigma)

def calculate_distance(PL):
    """
    与えられた PL [dBm] の値から距離 d [m] を計算します。
    """
    exponent = (PL_bar - PL + X_sigma) / (10 * gamma)
    d = d0 * (10 ** exponent)
    return d

# 例として PL = 50 [dBm] の場合の距離を計算
PL_input = 50.0  # 与えられた RSSI 値 [dBm]
distance = calculate_distance(PL_input)

print(f"測定されたRSSIが {PL_input} [dBm] のとき、距離 d は約 {distance:.2f} [m]")

測定されたRSSIが 50.0 [dBm] のとき、距離 d は約 0.37 [m]
